<img src="https://github.com/jina-ai/discoart/blob/main/.github/banner.png?raw=true" alt="Create Disco Diffusion artworks in one-line">
<p align=center>
<a href="https://pypi.org/project/discoart/"><img src="https://img.shields.io/pypi/v/discoart?style=flat-square&amp;label=Release" alt="PyPI"></a>
<a href="https://slack.jina.ai"><img src="https://img.shields.io/badge/Slack-3.1k-blueviolet?logo=slack&amp;logoColor=white&style=flat-square"></a>
<a href="https://colab.research.google.com/github/jina-ai/dalle-flow/blob/main/client.ipynb"><img src="https://img.shields.io/badge/Open-in%20Colab-brightgreen?logo=google-colab&style=flat-square" alt="Open in Google Colab"/></a>
</p>

😎 **If you are already a DD user**: you are ready to go! There is no extra learning, DiscoArt respects the same parameter semantics as DD5.2. So just unleash your creativity!

There are some minor differences between DiscoArt and DD5.x:
  - DiscoArt fixes multiple bugs e.g. weighted prompts, cut scheduling in original DD5.4, which improves the generation quality.
  - DiscoArt does not support video generation and `image_prompt` (which was marked as ineffective in DD 5.4).
  - Due to no video support, `text_prompts` in DiscoArt accepts a string or a list of strings, not a dictionary; i.e. no frame index `0:` or `100:`.
  - `clip_models` accepts a list of values from [all open-clip pretrained models and weights](https://github.com/jina-ai/discoart/blob/main/discoart/resources/docstrings.yml#L90).
  - `cut_ic_pow` is changed to a scheduling parameter to control the power of inner cut, the syntax is the same as `cut_overview`, `cut_innercut`. 

👶 **If you are a [DALL·E Flow](https://github.com/jina-ai/dalle-flow/) or new user**: you may want to take step by step, as Disco Diffusion works in a very different way than DALL·E. It is much more advanced and powerful: e.g. Disco Diffusion can take weighted & structured text prompts; it can initialize from a image with controlled noise; and there are way more parameters one can tweak. Impatient prompt like `"armchair avocado"` will give you nothing but confusion and frustration. I highly recommend you to check out the following resources before trying your own prompt:
- [Zippy's Disco Diffusion Cheatsheet v0.3](https://docs.google.com/document/d/1l8s7uS2dGqjztYSjPpzlmXLjl5PM3IGkRWI3IiCuK7g/mobilebasic)
- [EZ Charts - Diffusion Parameter Studies](https://docs.google.com/document/d/1ORymHm0Te18qKiHnhcdgGp-WSt8ZkLZvow3raiu2DVU/edit#)
- [Disco Diffusion 70+ Artist Studies](https://weirdwonderfulai.art/resources/disco-diffusion-70-plus-artist-studies/)
- [A Traveler’s Guide to the Latent Space](https://sweet-hall-e72.notion.site/A-Traveler-s-Guide-to-the-Latent-Space-85efba7e5e6a40e5bd3cae980f30235f#e122e748b86e4fc0ad6a7a50e46d6e10)
- [Disco Diffusion Illustrated Settings](https://coar.notion.site/Disco-Diffusion-Illustrated-Settings-cd4badf06e08440c99d8a93d4cd39f51)
- [Coar’s Disco Diffusion Guide](https://coar.notion.site/coar/Coar-s-Disco-Diffusion-Guide-3d86d652c15d4ca986325e808bde06aa#8a3c6e9e4b6847afa56106eacb6f1f79)

# Install

In [ ]:
!pip install -U discoart

from discoart import create


---

# Create artworks

In [ ]:
da = create()

That's it. It is that simple.

# Specify parameters

In [ ]:
from discoart import create

da = create(text_prompts='A painting of sea cliffs in a tumultuous storm, Trending on ArtStation.',
       init_image='https://d2vyhzeko0lke5.cloudfront.net/2f4f6dfa5a05e078469ebe57e77b72f0.png',
       skip_steps=100)


🎉 That's all you need to learn.

Of course if you have some extra time, it would be helpful for you to learn the following API of DiscoArt.


---

## Visualize results

`create()` returns `da`, a [DocumentArray](https://docarray.jina.ai/fundamentals/documentarray/)-type object. It contains the following information:
- All arguments passed to `create()` function, including seed, text prompts and model parameters.
- 4 generated image and its intermediate steps' images, where `4` is determined by `n_batches` and is the default value. 

This allows you to further post-process, analyze, export the results with powerful DocArray API.

Images are stored as Data URI in `.uri`, to save the first image as a local file:


In [ ]:
da[0].save_uri_to_file('discoart-result.png')

To save all final images:

In [ ]:
for idx, d in enumerate(da):
    d.save_uri_to_file(f'discoart-result-{idx}.png')

You can also display all four final images in a grid:

In [ ]:
da.plot_image_sprites(skip_empty=True, show_index=True, keep_aspect_ratio=True)

## Visualize intermediate steps

You can also zoom into a run and check out intermediate steps:

In [ ]:
da[0].chunks.plot_image_sprites(skip_empty=True, show_index=True, keep_aspect_ratio=True)

You can `.display()` the chunks one by one, or save one via `.save_uri_to_file()`, or save all intermediate steps as a GIF:

In [ ]:
da[0].chunks.save_gif('lighthouse.gif', show_index=True, inline_display=True, size_ratio=0.5)

## Export configs

You can review its parameters from `da[0].tags` or export it as an SVG image:

In [ ]:
from discoart.config import save_config_svg

save_config_svg(da, 'my.svg')

## Pull results anywhere anytime

At anytime on any machine, you can pull the real-time results (including paramters, intermedidate diffusion steps, final results) with a session ID:

> Please replace `discoart-3205998582` to your own when you run the above 2 cells!

In [ ]:
from docarray import DocumentArray

da = DocumentArray.pull('discoart-3205998582')

## Reuse a Document as initial state

Consider a Document as a self-contained data with config and image, one can use it as the initial state for the future run. Its `.tags` will be used as the initial parameters; `.uri` if presented will be used as the initial image.

In [ ]:
from discoart import create
from docarray import DocumentArray

da = DocumentArray.pull('discoart-3205998582')

create(init_document=da[0],
       cut_ic_pow=0.5,
       tv_scale=600, 
       cut_overview='[12]*1000', 
       cut_innercut='[12]*1000', 
       use_secondary_model=False)